---

In [50]:
# Correr antes : ipcluster start -n 2
compartido = []
from ipyparallel import Client
import time
rc = Client()

def funcion(param1, param2):
    from math import sqrt
    t = 0
    for x in range(40000000):
        t += sqrt(sqrt((x**2))**2)
    return t + param1 + param2

res1 = rc[0].apply(funcion,1,2)
res2 = rc[1].apply(funcion,4,5)
results = [res1, res2]

while not all(map(lambda ar: ar.ready(), results)):
    time.sleep(0.5)

compartido.append(res1.get())
compartido.append(res2.get())
print(compartido)

[799999980000003.0, 799999980000009.0]


---

# Imports

In [1]:
import pandas as pd
import numpy as np
from ipyparallel import Client
import time

# Entorno multitarea

In [2]:
cliente = Client()

---

# Carga de datos

In [3]:
auctions = pd.read_pickle("../../../../data/tp2/auctions_tp2_formateado.pkl")

---

# Armado de ventanas

In [5]:
for ventana_nro in range(1,6):
    auctions["ventana_{}".format(ventana_nro)] = (((17+ventana_nro) <= auctions["date"].dt.day) & ((auctions["date"].dt.day) <= (19+ventana_nro)))

In [11]:
compartido = []

def funcion(dia, dataset):
    import pandas as pd
    return (((17+dia) <= dataset["date"].dt.day) & ((dataset["date"].dt.day) <= (19+dia)))

res1 = cliente[0].apply(funcion,1, auctions)
res2 = cliente[1].apply(funcion,2, auctions)

while not all(map(lambda ar: ar.ready(), results)):
    time.sleep(0.5)

compartido.append(res1.get())
compartido.append(res2.get())

In [10]:
compartido[1]

10129864    False
25285906    False
3946062     False
3946063     False
24607726    False
11820552    False
44538105    False
4953159     False
11812696    False
11820553    False
47080629    False
44517641    False
44538106    False
3434618     False
44514683    False
44514684    False
44504337    False
16344775    False
10129865    False
16188706    False
44538107    False
20481863    False
16344776    False
16735299    False
11798280    False
44538108    False
11820554    False
20466501    False
14055018    False
44517642    False
            ...  
22779167    False
33429066    False
18430951    False
33102635    False
44059668    False
13744215    False
22778745    False
18430952    False
9498157     False
18430953    False
28212235    False
36347849    False
9498158     False
18430234    False
13743303    False
9498159     False
18431825    False
4298524     False
19909581    False
36347718    False
13743304    False
12712359    False
4301357     False
18430235    False
3801015   

---

# Creación de features

## Creación de sets de entrenamiento

In [4]:
Xs = {}
for ventana_nro in range(1,6):
    Xs[ventana_nro] = auctions.loc[auctions["ventana_{}".format(ventana_nro)]]["device_id"].drop_duplicates().to_frame().set_index("device_id")

## Función generadora de features

In [5]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos):
    for ventana_nro in range(1, 6):
        feature = generador_feature(dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]])
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, how="right", left_index=True, right_index=True)       

---

## Cantidad de apariciones en subastas dentro de la ventana

In [6]:
def cantidad_apariciones_en_subastas(dataframe):
    return dataframe.groupby("device_id").agg({"date" : "count"}).rename(index=str, columns={"date": "cantidad_apariciones"})

In [7]:
generar_feature_en_ventanas(auctions, cantidad_apariciones_en_subastas, Xs)

---

## Entrenamiento: [18/4 , 20/4] ; Target: [21/4 , 23/4]

---

## Entrenamiento: [19/4 , 21/4] ; Target: [22/4 , 24/4]

---

## Entrenamiento: [20/4 , 22/4] ; Target: [23/4 , 25/4]

---

## Entrenamiento: [21/4 , 23/4] ; Target: [24/4 , 26/4]

---

## Entrenamiento: [22/4 , 24/4] ; Target: [25/4 , 27/4]

---